Demonstrate making and applying calibration

In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
import Scan
soc = TopSoc()

In [ ]:
fMixer = soc.fsIn/2/2
soc.set_mixer(fMixer)
f0 = fMixer + 85.4
f1 = fMixer + 95.6
fTone = 0.5*(f0+f1)

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)
outCh = scan.soc.outFreq2ch(fTone)
print("measure nominal dealy for fTone=%f  outCh=%d"%(fTone, outCh))
nominalDelay = scan.measureNominalDelay(outCh, decimation=2, doProgress=True, doPlot=True)

In [ ]:
nf = 51
nf = 201
nt = 10
nt,nf = 1,51
nt,nf = 10,201
print("nominalDelay =",nominalDelay)
calibration = scan.makeCalibration(fMixer, f0, f1, nf=nf, nt=nt, doProgress=True,
                            nominalDelay=nominalDelay)
fscan = calibration['fscan']
print("delayApplied =",fscan['delayApplied'])


In [ ]:
iTone = 2
Scan.fscanPlot(fscan, iTone)


In [ ]:
spectrum = Scan.fscanToSpectrum(fscan)

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)
plt.plot(spectrum[0],spectrum[2])
print("f0 =",f0)
print("f1 =",f1)
fList = scan.makeFList(fMixer, f0, f1)
for f in fList:
    plt.axvline(f,color='r')
plt.plot([f0,f1],[0,0], 'g')

In [ ]:
sx = spectrum[1]*np.exp(1j*spectrum[2])
si = np.real(sx)
sq = np.imag(sx)
plt.plot(spectrum[0],si)
plt.plot(spectrum[0],sq)
for f in fList:
    plt.axvline(f,color='r')


In [ ]:
fMiddle = 0.5*(f0+f1)
fStart = fMiddle-4.3
fEnd = fMiddle+2.1
inds = (spectrum[0] > fStart ) & (spectrum[0] < fEnd)
plt.plot(spectrum[0][inds],si[inds], '.')
plt.plot(spectrum[0][inds],sq[inds], '.')
for f in fList:
    plt.axvline(f,color='r', alpha=0.1)
plt.xlim((fStart,fEnd))

In [ ]:

freq = 0.5*(fStart+fEnd)
iCalib = np.searchsorted(calibration['fList'], freq)-1
print("iCalib =",iCalib)
fa = calibration['fList'][iCalib]
fb = calibration['fList'][iCalib+1]
freqs = np.linspace(fa, fb)
xfit = calibration['cInterps'][iCalib](freqs)
plt.plot(freqs-fa, np.real(xfit), "o")
plt.plot(freqs-fa, np.imag(xfit), "o")
plt.xlabel("freq-%f"%fa)

In [ ]:
# Pretend we have a few frequencies we want to study
nTones = 4
testFreqs = fStart + 2.345*np.arange(nTones)

testAmps = 0.9*np.ones(len(testFreqs))/len(testFreqs)
np.random.seed(1234991)
testFis = np.random.uniform(0, 2*np.pi, size=len(testFreqs))
bandwidth = 0.4 # MHz
nf = 50
decimation = 2
nt = 3
doProgress = True

testFScan = scan.fscan(testFreqs, testAmps, testFis, bandwidth, nf, decimation,nt, doProgress=doProgress)


In [ ]:
Scan.fscanPlot(testFScan, 0)

In [ ]:
tfsCalib = Scan.applyCalibration(testFScan, calibration, amplitudeMax=5000)


In [ ]:
Scan.fscanPlot(tfsCalib, 0)